In [207]:
import numpy as np
import matplotlib
import os
import cv2
from skimage import io
from skimage import color, exposure, transform
from PIL import Image, ImageChops, ImageDraw, ImageOps, ImageFilter 
import imutils
import argparse
import ntpath

In [208]:
def load_templates(directory):
    paths = []
    for files in os.listdir(directory):
        if (files != ".DS_Store"):
            paths.append(directory+'/'+files)
    return paths

In [209]:
def manipulate_images(paths):
    
    for image_path in paths:
        
        
        
        img = Image.open(image_path)
        pix = img.load()
        
        
        borders = []
        roi_right = []
        roi_left = []
        
        for y in range(0,img.size[1]):
            
            found = False
            
            for x in range(1,img.size[0]):

                r = pix[x,y][0]
                g = pix[x,y][1]
                b = pix[x,y][2]
                
                prev_r = pix[x-1,y][0]
                prev_g = pix[x-1,y][1]
                prev_b = pix[x-1,y][2]
                
                if ((abs(prev_r-r)>=20) or (abs(prev_g-g)>=20) or (abs(prev_b-b)>=20)):
                    roi_right.append([x,y])
                    for i in range(0,x):
                        borders.append([i,y])
                    found = True
                    break
            
            
            
            if (found == False):
                for i in range(0,img.size[0]-1):
                    borders.append([i,y])
            
              
            for x in range(img.size[0]-1,1,-1):
                
                r = pix[x,y][0]
                g = pix[x,y][1]
                b = pix[x,y][2]
                
                prev_r = pix[x-1,y][0]
                prev_g = pix[x-1,y][1]
                prev_b = pix[x-1,y][2]
                
                if ((abs(prev_r-r)>=20) or (abs(prev_g-g)>=20) or (abs(prev_b-b)>=20)):
                    roi_left.append([x,y])
                    for i in range(x,img.size[0]-1):
                        borders.append([i,y])
                    break
         
        
        image = cv2.imread(image_path, -1)
        
        b_channel, g_channel, r_channel = cv2.split(image)
        alpha_channel = np.ones(b_channel.shape, dtype=b_channel.dtype) * 255 #creating a dummy alpha channel image.
        image_RGBA = cv2.merge((b_channel, g_channel, r_channel, alpha_channel))
        
        height, width, channels = image.shape
        
        #Deleting white perimeter of shape
        for i in range(0,img.size[0]-1):
            image_RGBA[0,i][3] = 0
            image_RGBA[img.size[1]-1,i][3] = 0
        
        for i in range(0,img.size[1]-1):
            image_RGBA[i,0][3] = 0
            image_RGBA[i,img.size[0]-1][3] = 0
            
        
        
        for border in borders:
            image_RGBA[border[1],border[0]][3] = 0
            
        
        head, tail = ntpath.split(image_path)
        
        title,extension = tail.split('.')
        cv2.imwrite("Traffic_Signs_Templates/Processed_Images/"+title+".png", image_RGBA)
        
        
        

In [210]:
directory = 'Traffic_Signs_Templates/Images'
paths = load_templates(directory)
manipulate_images(paths)